# Post-2024 Meme Coin Selection & Analysis

## Objective
Select high-quality meme coins launched after 2024-01-01 from the full 375M+ transaction dataset for comprehensive analysis.

## Selection Criteria
1. **First trading activity after 2024-01-01**
2. **Sufficient trading volume** (minimum thresholds)
3. **Active trading period** (not dead coins)
4. **Meme coin characteristics** (high volatility, community-driven)
5. **Data quality** (clean trading patterns)

## Expected Outcome
- Curated list of 10-20 high-quality post-2024 meme coins
- Initial characterization of each coin
- Foundation for comprehensive EDA framework

---


In [2]:
# Setup and imports
import sys
import os

# Add parent directory to path to import from root if needed
sys.path.append('.')
sys.path.append('..')

from solana_eda_utils import SolanaDataAnalyzer, format_large_number, truncate_address
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Initialize the analyzer
analyzer = SolanaDataAnalyzer()

print("✅ Connected to DuckDB database")
print(f"📊 Database: {analyzer.db_path}")
print(f"📋 Table: {analyzer.table_name}")
print(f"🎯 Target: Post-2024 meme coins selection")


Connected to database: /Volumes/Extreme SSD/DuckDB/solana.duckdb
✅ Connected to DuckDB database
📊 Database: /Volumes/Extreme SSD/DuckDB/solana.duckdb
📋 Table: first_day_trades
🎯 Target: Post-2024 meme coins selection


In [ ]:
# Export coin addresses for DexPaprika API lookup
print("\n💾 EXPORTING COIN ADDRESSES FOR DEXPAPRIKA LOOKUP")
print("=" * 60)

if post_2024_tokens is not None and len(post_2024_tokens) > 0:
    # Create a simple DataFrame with coin addresses and basic info for DexPaprika lookup
    coin_export_df = pd.DataFrame({
        'mint_address': post_2024_tokens['mint'],
        'first_trade_date': post_2024_tokens['first_trade_date'],
        'total_trades': post_2024_tokens['total_trades'],
        'unique_traders': post_2024_tokens['unique_traders'],
        'successful_trades': post_2024_tokens['successful_trades']
    })
    
    # Sort by total trades (most active first) for easier analysis
    coin_export_df = coin_export_df.sort_values('total_trades', ascending=False).reset_index(drop=True)
    
    # Save to CSV
    csv_filename = '../post2024_meme_coins_for_dexpaprika.csv'
    coin_export_df.to_csv(csv_filename, index=False)
    
    print(f"✅ Exported {len(coin_export_df):,} coin addresses to:")
    print(f"   📄 {csv_filename}")
    print(f"   📊 Sorted by trading activity (most active first)")
    print(f"   🔍 Ready for DexPaprika API enrichment")
    
    # Show preview of what was exported
    print(f"\n📋 PREVIEW OF EXPORTED DATA:")
    print("Top 10 most active coins:")
    for i, row in coin_export_df.head(10).iterrows():
        mint_short = truncate_address(row['mint_address'])
        print(f"{i+1:2d}. {mint_short} | {row['first_trade_date'].date()} | "
              f"{format_large_number(row['total_trades'])} trades | "
              f"{format_large_number(row['unique_traders'])} traders")
    
    print(f"\n💡 NEXT STEPS:")
    print(f"   1. Use the exported CSV with DexPaprika API")
    print(f"   2. Get token metadata (name, symbol, description)")
    print(f"   3. Merge back with trading data for enriched analysis")
    
else:
    print("❌ No post-2024 tokens available for export")


In [3]:
# Step 1: Find all tokens with first trading activity after 2024-01-01
print("🔍 STEP 1: IDENTIFYING POST-2024 TOKENS")
print("=" * 60)

# Query to get first trade date for each token
query_first_trades = """
SELECT 
    mint,
    MIN(block_timestamp) as first_trade_date,
    MAX(block_timestamp) as last_trade_date,
    COUNT(*) as total_trades,
    COUNT(DISTINCT swapper) as unique_traders,
    SUM(CASE WHEN succeeded THEN 1 ELSE 0 END) as successful_trades
FROM first_day_trades
GROUP BY mint
HAVING MIN(block_timestamp) >= '2024-01-01 00:00:00'
ORDER BY first_trade_date DESC
"""

print("Executing query to find post-2024 tokens...")
post_2024_tokens = analyzer.execute_query(query_first_trades)

if post_2024_tokens is not None and len(post_2024_tokens) > 0:
    print(f"✅ Found {len(post_2024_tokens):,} tokens that started trading after 2024-01-01")
    print(f"📊 Total trades from these tokens: {post_2024_tokens['total_trades'].sum():,}")
    print(f"👥 Total unique traders: {post_2024_tokens['unique_traders'].sum():,}")
    print(f"✅ Success rate: {post_2024_tokens['successful_trades'].sum() / post_2024_tokens['total_trades'].sum():.4f}")
    
    # Show first few examples
    print(f"\n📋 First 10 post-2024 tokens (newest first):")
    display_cols = ['mint', 'first_trade_date', 'total_trades', 'unique_traders']
    for i, row in post_2024_tokens.head(10).iterrows():
        mint_short = truncate_address(row['mint'])
        print(f"{i+1:2d}. {mint_short} | First: {row['first_trade_date'].date()} | "
              f"Trades: {format_large_number(row['total_trades'])} | "
              f"Traders: {format_large_number(row['unique_traders'])}")
else:
    print("❌ No post-2024 tokens found or query failed")


🔍 STEP 1: IDENTIFYING POST-2024 TOKENS
Executing query to find post-2024 tokens...
✅ Found 5,746 tokens that started trading after 2024-01-01
📊 Total trades from these tokens: 374,886,380
👥 Total unique traders: 43,232,161
✅ Success rate: 1.0000

📋 First 10 post-2024 tokens (newest first):
 1. EtmD8Bjd...PUMP | First: 2025-06-15 | Trades: 40.6K | Traders: 164
 2. 5Dh8TMLk...PUMP | First: 2025-06-15 | Trades: 53.8K | Traders: 214
 3. 2bkeRecU...pump | First: 2025-06-15 | Trades: 71.6K | Traders: 222
 4. 9KNtK6b8...pump | First: 2025-06-15 | Trades: 34.4K | Traders: 162
 5. 8J7Tygt5...pump | First: 2025-06-15 | Trades: 41.4K | Traders: 218
 6. 6ixcwjqf...Pump | First: 2025-06-15 | Trades: 36.5K | Traders: 159
 7. 5ykMDwJq...PUMP | First: 2025-06-15 | Trades: 34.2K | Traders: 173
 8. EjQLDvtg...Pump | First: 2025-06-15 | Trades: 59.1K | Traders: 200
 9. EPZEeKAE...PUMP | First: 2025-06-15 | Trades: 51.8K | Traders: 183
10. 39WSXpJs...PUMP | First: 2025-06-15 | Trades: 37.2K | Traders: 211

In [6]:
# Step 2: Analysis of first-day trading patterns (NO FILTERING)
print("\n📊 STEP 2: FIRST-DAY TRADING ANALYSIS")
print("=" * 60)

if post_2024_tokens is not None and len(post_2024_tokens) > 0:
    # Add calculated metrics for analysis
    post_2024_tokens['trading_hours'] = (post_2024_tokens['last_trade_date'] - post_2024_tokens['first_trade_date']).dt.total_seconds() / 3600
    post_2024_tokens['trades_per_trader'] = post_2024_tokens['total_trades'] / post_2024_tokens['unique_traders']
    post_2024_tokens['trades_per_hour'] = post_2024_tokens['total_trades'] / (post_2024_tokens['trading_hours'] + 0.1)  # Avoid division by 0
    
    print("📈 FIRST-DAY TRADING ANALYSIS:")
    print(f"Total post-2024 first-day meme coins: {len(post_2024_tokens):,}")
    print(f"Total first-day trades: {post_2024_tokens['total_trades'].sum():,}")
    print(f"Total unique first-day traders: {post_2024_tokens['unique_traders'].sum():,}")
    
    print(f"\n🔥 TRADING INTENSITY DISTRIBUTION:")
    print(f"  🏆 Most active token: {post_2024_tokens['total_trades'].max():,} trades")
    print(f"  📊 95th percentile: {post_2024_tokens['total_trades'].quantile(0.95):,.0f} trades") 
    print(f"  📊 75th percentile: {post_2024_tokens['total_trades'].quantile(0.75):,.0f} trades")
    print(f"  📊 50th percentile: {post_2024_tokens['total_trades'].median():,.0f} trades")
    print(f"  📊 25th percentile: {post_2024_tokens['total_trades'].quantile(0.25):,.0f} trades")
    
    print(f"\n👥 TRADER ENGAGEMENT:")
    print(f"  🏆 Most traders: {post_2024_tokens['unique_traders'].max():,} unique traders")
    print(f"  📊 95th percentile: {post_2024_tokens['unique_traders'].quantile(0.95):,.0f} traders")
    print(f"  📊 Median: {post_2024_tokens['unique_traders'].median():,.0f} traders")
    
    print(f"\n⚡ TRADING INTENSITY:")
    print(f"  🔥 Peak intensity: {post_2024_tokens['trades_per_hour'].max():,.0f} trades/hour")
    print(f"  📊 Median intensity: {post_2024_tokens['trades_per_hour'].median():,.0f} trades/hour")
    
    # No filtering - all tokens are valid first-day opportunities
    print(f"\n✅ ALL TOKENS INCLUDED:")
    print(f"🎯 Strategy: First-day trading opportunities")
    print(f"📋 Dataset: Already filtered to first-day trades only")
    print(f"🚀 Ready for volume analysis of all {len(post_2024_tokens):,} tokens")
    
    # Keep all tokens for next step
    filtered_tokens = post_2024_tokens.copy()
    
else:
    print("❌ No data available for analysis")



📊 STEP 2: FIRST-DAY TRADING ANALYSIS
📈 FIRST-DAY TRADING ANALYSIS:
Total post-2024 first-day meme coins: 5,746
Total first-day trades: 374,886,380
Total unique first-day traders: 43,232,161

🔥 TRADING INTENSITY DISTRIBUTION:
  🏆 Most active token: 1,111,649 trades
  📊 95th percentile: 215,521 trades
  📊 75th percentile: 85,818 trades
  📊 50th percentile: 41,187 trades
  📊 25th percentile: 13,467 trades

👥 TRADER ENGAGEMENT:
  🏆 Most traders: 161,259 unique traders
  📊 95th percentile: 28,481 traders
  📊 Median: 3,180 traders

⚡ TRADING INTENSITY:
  🔥 Peak intensity: 295,697 trades/hour
  📊 Median intensity: 5,754 trades/hour

✅ ALL TOKENS INCLUDED:
🎯 Strategy: First-day trading opportunities
📋 Dataset: Already filtered to first-day trades only
🚀 Ready for volume analysis of all 5,746 tokens
